In [27]:
run onehotencode.py

In [28]:
import sys
import os
import numpy as np
import tensorflow
import tensorflow as tf
import keras
import keras, keras.layers as L
sys.path.append("..")
from keras.models import load_model
from keras.layers import Input, Dense, Conv2D, LSTM, MaxPooling2D, UpSampling2D, BatchNormalization, Activation, Bidirectional
from keras.models import Model
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.optimizers import Adam
import os
import pickle
import numpy as np


import numpy as np
import pandas as pd


In [41]:
xtrain=np.array(xtrain)
img_shape=xtrain[0].shape
img_shape

(120, 56)

In [42]:
from keras_self_attention import SeqSelfAttention


In [43]:
def build_pca_autoencoder(img_shape,code_size=32):
    """
    Here we define a simple linear autoencoder as described above.
    We also flatten and un-flatten data to be compatible with image shapes
    """
    
    encoder = keras.models.Sequential()
    encoder.add(L.InputLayer(img_shape))
    encoder.add(L.Conv1D(filters=100, kernel_size=4, padding='same', activation='relu'))
    encoder.add(L.Bidirectional(LSTM(100,return_sequences=True)))
    encoder.add(L.Conv1D(filters=100, kernel_size=4, padding='same', activation='relu'))
    encoder.add(L.Bidirectional(LSTM(100,return_sequences=True)))
    encoder.add(L.Conv1D(filters=100, kernel_size=4, padding='same', activation='relu'))
    encoder.add(L.Bidirectional(LSTM(100,return_sequences=True)))
    encoder.add(L.Conv1D(filters=100, kernel_size=4, padding='same', activation='relu'))
    encoder.add(L.Bidirectional(LSTM(100,return_sequences=True)))
    encoder.add(L.Conv1D(filters=100, kernel_size=4, padding='same', activation='relu'))
    encoder.add(L.Bidirectional(LSTM(100,return_sequences=True)))
    encoder.add(SeqSelfAttention(attention_activation='tanh'))
    encoder.add(SeqSelfAttention(attention_activation='tanh'))
    encoder.add(SeqSelfAttention(attention_activation='tanh'))
    encoder.add(SeqSelfAttention(attention_activation='tanh'))
    encoder.add(L.Flatten())
    encoder.add(L.Dense(code_size))
    

    
    
    
    decoder = keras.models.Sequential()
    decoder.add(L.InputLayer((code_size,)))
    decoder.add(L.Dense(256))
    decoder.add(L.Dense(np.prod(img_shape)))#actual decoder, height*width*3 units
    decoder.add(L.Reshape(img_shape))#un-flatten
    decoder.add(SeqSelfAttention(attention_activation='tanh'))
    decoder.add(SeqSelfAttention(attention_activation='tanh'))
    decoder.add(SeqSelfAttention(attention_activation='tanh'))
    decoder.add(SeqSelfAttention(attention_activation='tanh'))
    return encoder,decoder

In [44]:
encoder,decoder = build_pca_autoencoder(img_shape,code_size=32)

inp = L.Input(img_shape)
code = encoder(inp)
reconstruction = decoder(code)

autoencoder = keras.models.Model(inp,reconstruction)
autoencoder.compile('adamax','mse')

In [45]:
autoencoder.fit(x=xtrain,y=xtrain,epochs=32)

Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Epoch 1/32
23/23 [==============================] - 11s 476ms/step - loss: 0.0056
Epoch 2/32
23/23 [==============================] - 2s 101ms/step - loss: 0.0055
Epoch 3/32
23/23 [==============================] - 3s 117ms/step - loss: 0.0054
Epoch 4/32
23/23 [==============================] - 3s 118ms/step - loss: 0.0098
Epoch 5/32
23/23 [==============================] - 2s 91ms/step - loss: 0.0054
Epoch 6/32
23/23 [==============================] - 2s 88ms/step - loss: 0.0055
Epoch 7/32
23/23 [==============================] - 2s 91ms/step - loss: 0.0055
Epoch 8/32
23/23 [==============================] - 2s 106ms/step - loss: 0.0055
Epoch 9/32
23/23 [==============================] - 2s 88ms/step - loss: 0.0055
Epoch 10/32
23/23 [==============================] - 4s 171ms/step - loss: 0.0055
Epoch 11/32
23/23 [==============================] - 3s 146ms/step

In [51]:
codes=encoder.predict(xtrain)

from sklearn.neighbors import LSHForest
lshf = LSHForest(n_estimators=50).fit(codes)


def get_similar(image, n_neighbors=2):

            
    code = encoder.predict(image[None])
    
    (distances,),(idx,) = lshf.kneighbors(code,n_neighbors=n_neighbors)
    
    return distances,xtrain[idx]

/opt/anaconda2/lib/python2.7/site-packages/sklearn/neighbors/approximate.py:258: DeprecationWarning: LSHForest has poor performance and has been deprecated in 0.19. It will be removed in version 0.21.
  DeprecationWarning)


In [56]:
get_similar(xtrain[0])[1]

array([1.1920929e-07, 2.3841858e-07], dtype=float32)